In [1]:
from torch.utils.data import DataLoader
from models import ADM, UDN
from methods import ADM_Deblur, UDN_Deblur, ADM_UDN_Combine
from datasets import Set14Dataset
from utils import ratio_determine
import torch
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
task = 'Set14'
sigma = 2.55 # selected from [2.55, 7.65, 12.75, 25.5]

adm_modelsigma = 49
adm_ite = 30
udn_modelsigma = 49

rho_list8 = sigma**2/np.logspace(np.log10(udn_modelsigma), np.log10(5), 8)**2
rho_list10 = sigma**2/np.logspace(np.log10(udn_modelsigma), np.log10(5), 10)**2


adm_ratio = ratio_determine(sigma)

adm_model = ADM().to(device).eval()

udn_model8 = UDN(rho_list8, n_layer = 8).to(device).eval()
udn_model8.load_state_dict(torch.load('models/UDN{}/model_{}_050.pth'.format(8, sigma))['stat_dict'])

udn_model10 = UDN(rho_list10, n_layer = 10).to(device).eval()
udn_model10.load_state_dict(torch.load('models/UDN{}/model_{}_050.pth'.format(10, sigma))['stat_dict'])

v = Set14Dataset(sigma = sigma)
V = DataLoader(dataset = v, num_workers = 1, batch_size = 1, shuffle = False)

for id, data in enumerate(V):
    adm_result = ADM_Deblur(adm_model, sigma, task_name = task, data = data, \
                        modelSigma1 = adm_modelsigma, modelSigma2 = 5, ite = adm_ite, early = -1, base = 30,\
                        u_thre = 0.05, l_thre = 0.05, ratio = adm_ratio)
    
    udn_result8 = UDN_Deblur(udn_model8, data, ratio = adm_ratio, n_layer = 8)
    udn_result10 = UDN_Deblur(udn_model10, data, ratio = adm_ratio, n_layer = 10)
    
    adm_udn_result = ADM_UDN_Combine(data, adm_result['Z-sub'][-1], udn_result8['Z-sub'][-1])
    adm_udn_result = ADM_UDN_Combine(data, adm_result['Z-sub'][-1], udn_result10['Z-sub'][-1])
    print('============================')

ADM deblur elapse time: 0.5323sec
task: test-g00-k00 blurry: 20.22/0.356 restored: 26.38/0.801
UDN deblur telapse time: 0.1608sec
task: test-g00-k00 blurry: 20.22/0.356 restored: 26.68/0.812
UDN deblur telapse time: 0.2046sec
task: test-g00-k00 blurry: 20.22/0.356 restored: 26.67/0.813
ADM_UDN deblur elapse time: 0.0083sec
task: test-g00-k00 blurry: 20.22/0.356 restored: 26.77/0.814
ADM_UDN deblur elapse time: 0.0061sec
task: test-g00-k00 blurry: 20.22/0.356 restored: 26.75/0.815
ADM deblur elapse time: 0.7925sec
task: test-g01-k01 blurry: 22.40/0.570 restored: 31.44/0.906
UDN deblur telapse time: 0.2497sec
task: test-g01-k01 blurry: 22.40/0.570 restored: 30.61/0.888
UDN deblur telapse time: 0.3157sec
task: test-g01-k01 blurry: 22.40/0.570 restored: 30.52/0.886
ADM_UDN deblur elapse time: 0.0102sec
task: test-g01-k01 blurry: 22.40/0.570 restored: 31.32/0.905
ADM_UDN deblur elapse time: 0.0101sec
task: test-g01-k01 blurry: 22.40/0.570 restored: 31.26/0.905
ADM deblur elapse time: 0.5245